In [2]:
#import of pandas and numpy
import pandas as pd
import numpy as np

In [17]:
#imports dataset

marketing_demo_raw_data = pd.read_csv("Datasets/marketing_demographics.csv")
marketing_demo_raw_data

,ID,Education,Marital_Status,Income,Country,Age,Children
0,0,Graduation,Married,70951.0,SA,37,0
1,1,Graduation,Single,57091.0,CA,61,0
2,9,Master,Single,46098.0,CA,47,2
3,13,PhD,Widow,25358.0,SA,75,1
4,17,PhD,Married,60491.0,SP,51,1
...,...,...,...,...,...,...,...
2211,11178,Master,Single,42394.0,GER,50,1
2212,11181,PhD,Married,156924.0,CA,73,0
2213,11187,Basic,Single,26487.0,CA,44,1
2214,11188,Graduation,Together,26091.0,SP,65,2


In [15]:
#imports dataset
marketing_buss_raw_data = pd.read_csv("Datasets/marketing_business.csv")
marketing_buss_raw_data

,ID,Total_Amount,Total_Purchase,Web_Purchase,Store_Purchase,Accepted_Campaigns,Website_Visit,Complaint,Response
1491,0,1198,16,3,13,0,1,0,0
1,1,577,17,7,10,1,5,0,1
1937,9,120,7,3,4,0,8,0,0
1300,13,32,4,1,3,0,6,0,0
1818,17,1028,24,6,18,0,5,0,0
...,...,...,...,...,...,...,...,...,...
1541,11178,32,4,1,3,0,7,0,0
1906,11181,8,0,0,0,0,0,0,0
524,11187,69,6,2,4,0,5,0,0
1878,11188,89,6,2,4,0,5,0,0


In [115]:
#join datasets on ID
marketing_raw_data= marketing_buss_raw_data.set_index("ID").join(marketing_demo_raw_data.set_index("ID"))

# Task 1.1
Calculate the average of ’Total Amount’ and ’Total Purchase’ for each Education category. Do you
find the same category to have the highest average for both amount and purchase?

Answer: YES, PhD has the highest both in Amount and Purchase

In [139]:
task1_1_data = marketing_raw_data[["Total_Amount","Total_Purchase","Education"]]

In [140]:
#check for nan values
print(task1_1_data.isnull().sum().any())

False


In [141]:
task1_1_data.groupby("Education").mean()

,Total_Amount,Total_Purchase
Education,,
2n Cycle,494.930000,11.680000
Basic,81.796296,5.222222
Graduation,621.686380,12.695341
Master,609.767123,12.468493
PhD,676.733888,13.492723


# Task 1.2
Filter your data for countries from which there is at least 200 customers. From which country in this
filtered dataset you can find the most complaints (column Complaint)

# Answer:
SP = Spain?

In [142]:
task1_2_data = marketing_raw_data.loc[:, ("Complaint","Country")]
print(task1_2_data.isnull().sum().any())

False


In [143]:
complaint_counts = task1_2_data.groupby("Country")["Complaint"].transform("count")
mask = complaint_counts >= 200
new_df = task1_2_data[mask]
print(new_df.groupby("Country")["Complaint"].sum())


Country
CA     2
SA     3
SP    14
Name: Complaint, dtype: int64


# Task 1.3
Calculate a new column that is 0, if the customer has 0 children, and 1 otherwise (so if the customer
has at least 1 child). Do customers with or without children have higher average Income? Check
whether the difference is statistically significant using a t-test!

# Answer:
People without children have an higher income.
The difference is statistically signigicant as the T value is -16.907 which indicates there is a significant difference between income without children then with children. Also as our p-test value for our null hypothesis "There is no difference between customers with or without kids" is pretty much zero, so there is strong evidence that our findings.

In [144]:
task1_3_data =  marketing_raw_data.loc[:, ("Children", "Income")]
print(task1_3_data.isnull().sum().any())

False


In [145]:
task1_3_data["HasChildren"] = task1_3_data["Children"].apply(lambda x: 0 if x == 0 else 1)
print(task1_3_data.groupby("HasChildren")["Income"].mean())

HasChildren
0    65705.565561
1    46865.626027
Name: Income, dtype: float64


In [146]:
# t-test import scipys own t test
from scipy.stats import ttest_ind

income_with_children = task1_3_data.loc[task1_3_data["HasChildren"] == 1, "Income"]
income_without_children = task1_3_data.loc[task1_3_data["HasChildren"] == 0, "Income"]
t, p = ttest_ind(income_with_children, income_without_children)
print("t-value: ", t)
print("p-value: ", p)

t-value:  -16.9071717250489
p-value:  2.08492231266765e-60


# Task 1.4
The company believes that their main target group is the customers aged between 18 and 45 years.
To check whether it is reflected in the sales, check whether the average number of Web Purchase is
higher for the customers in the target age group than for other customers. Do you have the same
result when you use Store Purchase? (Hint: you can create a new column that is 1 when the customer
is in the target age group and 0 otherwise).

# Answer:
The not targeted group has a higher average amount of number of Web Purchases.

The not targeted group has a higher average amount of number of Store Purchases.

In [147]:
task1_4_data =  marketing_raw_data.loc[:, ("Store_Purchase","Web_Purchase", "Age")]
print(task1_4_data.isnull().sum().any())

False


In [148]:

task1_4_data["Age_target"] = task1_4_data["Age"].apply(lambda x: 1 if 18<=x<=45 else 0)
# Web_purchase
print(task1_4_data.groupby("Age_target")["Web_Purchase"].mean())
# Store_purchase
print(task1_4_data.groupby("Age_target")["Store_Purchase"].mean())

Age_target
0    4.293317
1    3.525000
Name: Web_Purchase, dtype: float64
Age_target
0    8.762995
1    7.688333
Name: Store_Purchase, dtype: float64


# Task 1.5
Calculate the correlation between the columns Total Purchase, Age, and Income. Which of the
other two variables seem to be more related to Total Purchase? Based on this and the previous
question, do you think it would be useful for the company to focus a lot of efforts on age-based
segmentation/marketing?
# Answer:
Income and Total_Purchase has a moderate positiv correlation while Age and Total_Purchases has only a very week positiv correlation. So I would not focus my efforts on age-based marketing. 


In [166]:
task1_5_data =  marketing_raw_data.loc[:, ("Total_Purchase","Age", "Income")]

print(task1_5_data.isnull().sum().any())

False


In [167]:
print(task1_5_data.corr())


                Total_Purchase       Age    Income
Total_Purchase        1.000000  0.165397  0.625755
Age                   0.165397  1.000000  0.161791
Income                0.625755  0.161791  1.000000
